In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df=pd.read_csv('titanic.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [7]:
X_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
340,2,male,2.00,1,1,26.00,S
369,1,female,24.00,0,0,69.30,C
543,2,male,32.00,1,0,26.00,S
403,3,male,28.00,1,0,15.85,S
755,2,male,0.67,1,1,14.50,S


In [8]:
y_train.sample(4)

45     0
438    0
377    0
677    1
Name: Survived, dtype: int64

In [9]:
#1-pipeline.step fill the missing values by using SimpleImputer
ctrf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [10]:
#2-pipeline.step one hot coding 
ctrf2=ColumnTransformer([
    ('one_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
     ],remainder='passthrough')


In [11]:
#3.-pipeline.step scaling on feature
ctrf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [12]:
#4-Pipeline.step feature scaling
ctrf4=SelectKBest(score_func=chi2,k=6)

In [13]:
#5-Pipeline.step model
ctrf5=DecisionTreeClassifier()

## Create Join Pipeline

In [14]:
pipe=Pipeline([
    ('ctrf1',ctrf1),
    ('ctrf2',ctrf2),
    ('ctrf3',ctrf3),
    ('ctrf4',ctrf4),
    ('ctrf5',ctrf5)
])

## Pipeline vs Make_Piple 

#Aternate step to Pipe
pipe=make_pipeline(ctrf1,ctrf2,ctrf3,ctrf4,ctrf5)

In [15]:
#Display the Pipeline model
from sklearn import set_config
set_config(display='diagram')

In [16]:
#train 
pipe.fit(X_train,y_train)

Pipeline(steps=[('ctrf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('ctrf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('ctrf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('ctrf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x0000021B871B93A0>)),
                ('ctrf5', DecisionTreeClassifier())])

## Explore the Pipeline

In [17]:
pipe.named_steps

{'ctrf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'ctrf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('one_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'ctrf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'ctrf4': SelectKBest(k=6, score_func=<function chi2 at 0x0000021B871B93A0>),
 'ctrf5': DecisionTreeClassifier()}

In [18]:
#predict
y_pred=pipe.predict(X_test)

In [19]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

## Cross Validation using Pipeline

In [21]:
#Cross Validation using cross _val _score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6391214419383433

## Grid Search Using Pipeline

In [24]:
#to adjust the hyper parameter tunning to increase the accuracy of the algorithm
#grid search
params={
    'ctrf5__max_depth':[1,2,3,4,5,None]
}

In [25]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ctrf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('ctrf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('ctrf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('ctrf4',
                                        SelectKBest(k=6,
                                                    score_func=<function chi2 at 0x0000021B871B93A0>)),
                                       ('ctrf5', DecisionTreeClassifier())]),
             param_grid={'ctrf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [26]:
grid.best_score_

0.6391214419383433

In [27]:
grid.best_params_

{'ctrf5__max_depth': 2}

## Exporting The Pipeline


In [30]:
#Export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))